<a name='1'></a>
## 1 - Packages

Run the cell below to import all the libraries you'll need:

In [1]:
import tensorflow as tf
import numpy as np
import tensorflow.keras.layers as tfl

In [2]:
def ContBlock(X, filters_size=32, dropout_rate=0, pooling=True):

    X = tfl.Conv2D(filters = filters_size, kernel_size = 3, activation='relu', padding='same')(X)
    X = tfl.Conv2D(filters = filters_size, kernel_size = 3, activation='relu', padding='same')(X)
    
    if dropout_rate > 0:
        X = tfl.Dropout(rate = dropout_rate)(X)

    skip_connection = X
    
    if pooling:
        X = tfl.MaxPooling2D(pool_size = (2,2))(X)
    
    return X, skip_connection

In [3]:
def ExpBlock(X_exp, X_cont, filters_size=32):

    X_exp = tfl.Conv2DTranspose(filters = filters_size, kernel_size = 3, strides=2, padding='same')(X_exp)

    X = tfl.concatenate([X_exp, X_cont], axis=3)
    X = tfl.Conv2D(filters = filters_size, kernel_size = 3, activation='relu', padding='same')(X)
    X = tfl.Conv2D(filters = filters_size, kernel_size = 3, activation='relu', padding='same')(X)
 
    return X

In [4]:
def Unet(input_size, filters_size, classes):

    X_inputs = tfl.Input(input_size)
    
    # contracting path
    cb1 = ContBlock(X_inputs,   filters_size * 1)
    cb2 = ContBlock(cb1[0], filters_size * 2)
    cb3 = ContBlock(cb2[0], filters_size * 4)
    cb4 = ContBlock(cb3[0], filters_size * 8 , dropout_rate= 0.2)
    cb5 = ContBlock(cb4[0], filters_size * 16, dropout_rate= 0.2, pooling=False) 

    
    # expansive path
    eb1 = ExpBlock(cb5[0], cb4[1],  filters_size * 8)
    eb2 = ExpBlock(eb1,    cb3[1],  filters_size * 4)
    eb3 = ExpBlock(eb2,    cb2[1],  filters_size * 2)
    eb4 = ExpBlock(eb3,    cb1[1],  filters_size * 1)


    X = tfl.Conv2D(filters = filters_size, kernel_size = 3, activation='relu', padding='same')(eb4)
    
    X_output = tfl.Conv2D(filters = classes, kernel_size = 1, padding='same')(X)
    
    model = tf.keras.Model(inputs=X_inputs, outputs=X_output, name = 'Unet')

    return model

<a name='3-5'></a>
### 3.5 - Set Model Dimensions

In [8]:
input_size = (128,128,3)
filters_size = 32
classes =10   # number of possible labels for each pixel

unet = Unet(input_size = input_size, filters_size = filters_size, classes = classes)

### Check out the model summary below! 

In [9]:
unet.summary()

Model: "Unet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d_20 (Conv2D)              (None, 128, 128, 32) 896         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_21 (Conv2D)              (None, 128, 128, 32) 9248        conv2d_20[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, 64, 64, 32)   0           conv2d_21[0][0]                  
_______________________________________________________________________________________________